# Dataset Exploration — `test_customers.csv`

This notebook performs a focused exploratory data analysis (EDA) of the synthetic customer churn dataset `test_customers.csv`.

Goals for this notebook:
- Validate schema and types, inspect distributions and missing values.
- Detect anomalies and outliers.
- Explore relationships between features and the `churn` target.
- Produce clear visualizations (histograms, boxplots, correlation heatmap) and a concise business summary comparing churners vs non-churners.

Run the cells sequentially. The notebook is written to be reproducible and presentation-ready for an interview.

In [ ]:
# Section: Imports, settings and plotting style
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
from scipy import stats
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

# Display and plotting settings
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', lambda x: f"{x:,.2f}")

sns.set(style='whitegrid', palette='deep', font_scale=1.0)
plt.rcParams['figure.figsize'] = (10, 5)
pio.renderers.default = 'notebook_connected'

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print('Imports and settings completed')

In [ ]:
# Section: Load dataset, validate schema, quick peek
DATA_PATH = os.path.join('generate', 'test_customers.csv')  # relative to notebook in src/synthetic_dataset
assert os.path.exists(DATA_PATH), f"Dataset not found at {DATA_PATH}"

# Load dataframe
df = pd.read_csv(DATA_PATH, low_memory=False)
print(f"Loaded dataframe with shape: {df.shape}")

# Quick peek
display(df.head())
print('\nDataFrame info:')
df.info()

# Programmatic column assertions (derived from assignment)
expected_cols = [
    'customer_id','age','gender','city_population','household_size','annual_income','education_level',
    'employment_status','tenure_months','main_account_type','number_of_accounts','num_standard_accounts',
    'num_savings_accounts','num_student_accounts','num_business_accounts','number_of_debit_cards','number_of_credit_cards',
    'total_loan_amount','number_of_loans','current_balance','days_from_last_transaction','avg_monthly_balance_3m',
    'avg_monthly_balance_12m','avg_monthly_spend_3m','avg_monthly_spend_12m','transaction_count_3m','transaction_count_12m',
    'transaction_volume_3m','transaction_volume_12m','declined_transactions_3m','declined_transactions_12m','credit_limit',
    'revolving_balance','credit_utilization_ratio','loan_payments_3m','loan_payments_12m','complaints_3m','complaints_12m',
    'support_contacts_3m','support_contacts_12m','last_support_contact_days','received_satisfaction_survey','filled_satisfaction_survey',
    'nps_segment','days_from_last_login','days_from_last_web_login','days_from_last_app_login','mobile_app_logins_3m','mobile_app_logins_12m',
    'online_banking_logins_3m','online_banking_logins_12m','avg_time_spent_per_login_minutes_3m','avg_time_spent_per_login_minutes_12m',
    'push_clicks_3m','push_clicks_12m','marketing_emails_opened_3m','marketing_emails_opened_12m','churn'
]
missing = [c for c in expected_cols if c not in df.columns]
assert not missing, f"Missing columns: {missing}"
print('All expected columns are present')

# Basic duplicates / key checks
assert df['customer_id'].is_unique, 'customer_id contains duplicates'
print('customer_id uniqueness OK')

# Convert obvious boolean-like columns
for bool_col in ['received_satisfaction_survey','filled_satisfaction_survey','churn']:
    if bool_col in df.columns:
        # Some CSVs may have True/False strings; coerce
        df[bool_col] = df[bool_col].replace({'True': True, 'False': False}).astype('bool')

# Convert nps_segment to categorical
if 'nps_segment' in df.columns:
    df['nps_segment'] = df['nps_segment'].astype('category')

print('\nSample of dtypes after conversions:')
print(df.dtypes.head(18))

# Basic statistics
print('\nNumeric summary (selected):')
df.select_dtypes(include=[np.number]).describe().T[['count','mean','std','min','25%','50%','75%','max']]


In [ ]:
# Section: Missing values audit, univariate distributions, correlations, and churn comparisons
import matplotlib.ticker as mtick

# 1) Missing values summary
missing_counts = df.isna().sum().sort_values(ascending=False)
missing_pct = (missing_counts / len(df) * 100).round(2)
missing_df = pd.DataFrame({'missing_count': missing_counts, 'missing_pct': missing_pct})
missing_df.head(20)

# Visualize missingness top 30
plt.figure(figsize=(10,6))
sns.barplot(x=missing_df['missing_pct'].head(30).values, y=missing_df.index[:30], color='steelblue')
plt.xlabel('% missing')
plt.title('Top 30 columns by missing %')
plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter())
plt.show()

# 2) Quick numeric distributions (select key numeric columns)
num_cols = [
    'age','annual_income','tenure_months','current_balance','avg_monthly_spend_3m',
    'transaction_count_3m','mobile_app_logins_3m','avg_time_spent_per_login_minutes_3m'
]
num_cols = [c for c in num_cols if c in df.columns]

for c in num_cols:
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    sns.histplot(df[c].dropna(), kde=True, color='cornflowerblue', bins=40)
    plt.title(f'Histogram + KDE: {c}')

    plt.subplot(1,2,2)
    sns.boxplot(x=df[c], color='lightcoral')
    plt.title(f'Boxplot: {c}')
    plt.tight_layout()
    plt.show()

# 3) Log transform for heavy-tailed features (example)
heavy = ['annual_income','transaction_volume_12m','avg_monthly_spend_3m']
heavy = [c for c in heavy if c in df.columns]
for c in heavy:
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    sns.histplot(df[c].dropna(), kde=True, bins=40)
    plt.title(f'Raw: {c}')
    plt.subplot(1,2,2)
    sns.histplot(np.log1p(df[c].dropna()), kde=True, bins=40, color='seagreen')
    plt.title(f'log1p: {c}')
    plt.tight_layout()
    plt.show()

# 4) Correlation heatmap (numeric features subset)
num_df = df.select_dtypes(include=[np.number]).copy()
# sample or limit columns to keep heatmap readable
corr_cols = num_df.columns.tolist()
if len(corr_cols) > 40:
    corr_cols = num_df.corr().abs().mean().sort_values(ascending=False).head(40).index.tolist()

corr = df[corr_cols].corr()
plt.figure(figsize=(12,10))
sns.heatmap(corr, cmap='coolwarm', center=0, annot=False, fmt='.2f')
plt.title('Correlation heatmap (selected numeric features)')
plt.show()

# 5) Churn target basic stats
print('Churn distribution:')
print(df['churn'].value_counts(dropna=False))
print('\nChurn rate: {:.2f}%'.format(df['churn'].mean()*100))

# 6) Compare numeric features by churn (boxplots and group stats)
churn_groups = df.groupby('churn')
summary_by_churn = churn_groups[num_cols].agg(['count','mean','median','std']).T
summary_by_churn.head()

for c in num_cols:
    plt.figure(figsize=(8,4))
    sns.boxplot(x='churn', y=c, data=df, palette=['#5ab4ac','#d8b365'])
    plt.title(f'{c} by churn')
    plt.show()

# 7) Categorical feature churn rates (selected)
cat_cols = ['gender','main_account_type','education_level','nps_segment']
cat_cols = [c for c in cat_cols if c in df.columns]

for c in cat_cols:
    ct = pd.crosstab(df[c], df['churn'], normalize='index')
    ct = ct.sort_values(by=True, ascending=False)
    display(ct.head(10))
    ct.plot(kind='bar', stacked=True, figsize=(8,4), color=['#5ab4ac','#f08080'])
    plt.title(f'Churn rate by {c}')
    plt.ylabel('Proportion')
    plt.legend(title='churn')
    plt.show()

# 8) Quick statistical tests for a few numeric features (t-test)
from scipy.stats import ttest_ind
print('\nT-tests (churn=True vs churn=False)')
for c in ['age','annual_income','tenure_months','avg_monthly_spend_3m','transaction_count_3m']:
    if c in df.columns:
        a = df.loc[df['churn']==True, c].dropna()
        b = df.loc[df['churn']==False, c].dropna()
        if len(a)>10 and len(b)>10:
            tstat, pval = ttest_ind(a, b, equal_var=False)
            print(f"{c}: t={tstat:.3f}, p={pval:.3e}, mean(churn)={a.mean():.2f}, mean(no-churn)={b.mean():.2f}")

# 9) Save a small cleaned snapshot for downstream work
out_path = os.path.join('..', 'generate', 'test_customers_cleaned_sample.csv')
df.to_csv(out_path, index=False)
print(f'Wrote cleaned snapshot to {out_path}')

# 10) Brief business-oriented summary (print)
print('\n--- EXECUTIVE SUMMARY (placeholder) ---')
print('Run the notebook to produce visuals. Key comparisons are displayed above: numeric distributions, correlation heatmap, and churn-group differences.')
print('Next steps: formalize feature engineering, build segmentation, and train models as per assignment.')
